In [0]:
%sql
USE CATALOG main;
USE SCHEMA demo;

SHOW TABLES IN demo;

In [0]:
%sql
-- sanity: row samples
SELECT siteId, ingestRunId, tileX, tileY, waterRatio, tileSizeM, isMostlyWater
FROM demo_agg_tile_water_mask
WHERE siteId='wellington_cbd'
LIMIT 20;

In [0]:
%sql

-- hottest tiles by density
SELECT count(distinct tileX)
FROM demo_agg_tile_stats
WHERE siteId='wellington_cbd'
-- ORDER BY pointCount DESC
LIMIT 20;


In [0]:
%sql
SELECT surfaceId, surfaceType, tileX, tileY, minX, minY, maxX, maxY, cellsUri, cellSizeM, tileSizeM
FROM demo.demo_agg_surface_patches
WHERE siteId='wellington_cbd'
  AND ingestRunId='20130101'
  AND surfaceType IN ('ground','water')
  AND maxX >= :qMinX AND minX <= :qMaxX
  AND maxY >= :qMinY AND minY <= :qMaxY
ORDER BY surfaceType, tileX, tileY
LIMIT 200;

In [0]:
%sql
DESCRIBE demo_agg_surface_cells

In [0]:
%sql
SELECT siteId, ingestRunId, surfaceType, tileX, tileY, cellX, cellY, z
FROM demo.demo_agg_surface_cells
WHERE siteId='wellington_cbd'
  AND ingestRunId='20130101'
  AND surfaceType='ground'
  AND cellMaxX >= :qMinX AND cellMinX <= :qMaxX
  AND cellMaxY >= :qMinY AND cellMinY <= :qMaxY
LIMIT 5000;

In [0]:
# Re-register all tables (data files still exist in ADLS)
print("=== Re-registering demo schema and tables ===")

# 1. Create schema
spark.sql("""
    CREATE SCHEMA IF NOT EXISTS demo
    LOCATION 'abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/schemas/demo'
""")
print("✅ demo schema")

# 2. Register all tables
tables_config = [
    ("demo_processed_points_tiled", "abfss://processed@trimblegeospatialdemo.dfs.core.windows.net/points_tiled"),
    ("demo_agg_tile_stats", "abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/tile_stats"),
    ("demo_agg_tile_water_mask", "abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/tile_water_mask"),
    ("demo_agg_surface_cells", "abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/surfaces/surface_cells"),
    ("demo_agg_surface_patches", "abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/surfaces/surface_patches"),
]

for table_name, location in tables_config:
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS demo.{table_name}
        USING DELTA
        LOCATION '{location}'
    """)
    print(f"✅ demo.{table_name}")

print("\n=== Verification ===")
spark.sql("SHOW TABLES IN demo").show(truncate=False)

print("\n✅ All tables re-registered! You can now run your queries.")
print("\n💡 Tip: Data files were never deleted - we just reconnected the metastore to them.")

In [0]:
%sql
-- Summary by surface type
SELECT surfaceType, COUNT(*) AS patchCount, SUM(pointCountUsed) AS pointsUsed
FROM demo.demo_agg_surface_patches
WHERE siteId='wellington_cbd' AND ingestRunId='20130101'
GROUP BY surfaceType;

In [0]:
%sql
-- Top patches by point count
SELECT *
FROM demo.demo_agg_surface_patches
WHERE siteId='wellington_cbd' AND ingestRunId='20130101'
ORDER BY pointCountUsed DESC
LIMIT 20;

In [0]:
%sql
USE CATALOG main;
USE SCHEMA demo;
-- DESCRIBE surface_cells_v2;

SELECT * 
FROM surface_cells_v2
Limit(20)